In [1]:
import tensorflow as tf

In [2]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
sys.path.insert(0,'lucashnegri-peakutils-51a679cd8428')
# sys.path.insert(0,'S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
# from models import *
from Generator0 import *
from keras.preprocessing import sequence
# import peakutils
from scipy import special
import scipy.io as sio
from keras.preprocessing.sequence import pad_sequences
import peakutils
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint


In [3]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [4]:
# data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/jk21/Downloads/Data/data/murmur/train'
test_folder = '/home/jk21/Downloads/Data/data/murmur/test'


In [5]:
model_folder = 'lcnn2'

In [6]:
model_folder

'lcnn2'

In [7]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [8]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [9]:
winlen = 512
hoplen = 256
nmel = 120 
nsec = 20 
trim = 1 
use_mel=True
use_cqt = False 
use_stft = False
use_raw=False
use_interval=True
use_wav2=True
maxlen1 = 246000
min_dist = 500
max_interval_len=229
per_sec=4000

In [10]:
model1_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_folder,
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    options=None,
    initial_value_threshold=None
)


In [11]:
model2_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_folder,
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    options=None,
    initial_value_threshold=None
)    

In [12]:
params_feature = {'samp_sec': nsec,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': hoplen,
                  'win_length': winlen,
                  'n_mels': nmel,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10,
                  'maxlen1': maxlen1,
                  'min_dist':min_dist,
                  'max_interval_len' : max_interval_len,
                  'trim' :1,
                  'use_mel' : use_mel,
                  'use_cqt' : use_cqt,
                  'use_stft' : use_stft,
                  'use_interval' : use_interval,
                  'use_wav2' : use_wav2,
                  'per_sec'  : 4000
                 }

In [13]:
features_trn, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape,wav2_input_shape = get_features_3lb_all_ord(train_folder, patient_files_trn, **params_feature)


100%|█████████████████████████████████████████| 751/751 [15:20<00:00,  1.23s/it]


melspec:  120 313
cqt:  1 1
stft:  1 1
interval:  229 1
wav2:  (246000,)


In [15]:
features_test, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape,wav2_input_shape= get_features_3lb_all_ord(test_folder, patient_files_test, **params_feature)

100%|█████████████████████████████████████████| 191/191 [04:38<00:00,  1.46s/it]


melspec:  120 313
cqt:  1 1
stft:  1 1
interval:  229 1
wav2:  (246000,)


In [22]:
len(features_trn['id'])

2532

In [19]:
len(features_trn)

14

In [14]:
with open('/home/jk21/Downloads/Data/features_trn_1sec_interval.pickle', 'wb') as f:
    pickle.dump(features_trn, f, pickle.HIGHEST_PROTOCOL)

In [16]:
with open('/home/jk21/Downloads/Data/features_test_1sec_interval.pickle', 'wb') as f:
    pickle.dump(features_test, f, pickle.HIGHEST_PROTOCOL)

In [17]:
with open('/home/jk21/Downloads/Data/features_trn_1sec_interval.pickle', 'rb') as f:
    features_trn = pickle.load(f)

In [18]:
with open('/home/jk21/Downloads/Data/features_test_1sec_interval.pickle', 'rb') as f:
    features_test = pickle.load(f)

In [23]:
mm_weight = 3 #np.random.choice([2,3,4,5])
oo_weight = 3 #np.random.choice([2,3,4,5,6])
ord1 = True #np.random.choice([True,False])
mm_mean = False #np.random.choice([True,False])
dp = 0 #np.random.choice([0, .1, .2, .3])
fc = False #np.random.choice([True,False])
ext = True
chaug = 10 #np.random.choice([0, 10])
mixup = True #np.random.choice([True,False])
cout = .8 #np.random.choice([0, 0.8])
wunknown = 1 #np.random.choice([1, 0.7, .5, .2])
n1 = 0 #np.random.choice([0,2])
if n1 == 0 :
    ranfil = False
else :
    ranfil = [n1, [18,19,20,21,22,23]]

In [24]:
params_feature['ord1'] = ord1
params_feature['mm_mean'] = mm_mean
params_feature['dp'] = dp
params_feature['fc'] = fc
params_feature['ext'] = ext
params_feature['oo_weight'] = oo_weight
params_feature['mm_weight'] = mm_weight
params_feature['chaug'] = chaug
params_feature['cout'] = cout
params_feature['wunknown'] = wunknown
params_feature['mixup'] = mixup
params_feature['n1'] = n1

In [25]:
mel_input_shape = features_trn['mel1'].shape[1:]

In [26]:
mel_input_shape

(120, 313, 1)

In [27]:
cqt_input_shape = features_trn['cqt1'].shape[1:]

In [28]:
cqt_input_shape

(1, 1, 1)

In [29]:
stft_input_shape = features_trn['stft1'].shape[1:]

In [30]:
stft_input_shape

(1, 1, 1)

In [31]:
interval_input_shape= features_trn['interval'].shape[1:]

In [32]:
interval_input_shape

(229, 1)

In [33]:
wav2_input_shape= features_trn['wav2'].shape[1:]

In [34]:
wav2_input_shape

(246000,)

In [35]:
for i in range(len(features_trn['interval'])):
    features_trn['interval'][i]=np.array(features_trn['interval'][i])

In [36]:
for i in range(len(features_trn['mel1'])):
    features_trn['mel1'][i]=np.array(features_trn['mel1'][i])

In [37]:
import tensorflow as tf
import tensorflow_hub as hub

wav2vec2_pretrained = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2/1", trainable=False)
# For using this model, it's important to set `jit_compile=True` on GPUs/CPUs
# as some operations in this model (i.e. group-convolutions) are unsupported without it


2022-08-09 16:58:37.235877: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-09 16:58:37.239369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-09 16:58:37.239666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-09 16:58:37.240127: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [38]:
inputs = tf.keras.Input(shape=(246000,))
hidden_states = wav2vec2_pretrained(inputs)

In [39]:
wav2vec2_model = keras.Model(inputs = inputs , outputs = hidden_states )

In [40]:
wav2vec2_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 246000)]          0         
                                                                 
 keras_layer (KerasLayer)    (None, 768, 768)          94371712  
                                                                 
Total params: 94,371,712
Trainable params: 0
Non-trainable params: 94,371,712
_________________________________________________________________


In [41]:
@tf.function(jit_compile=True)
def forward(speech):
    return wav2vec2_model(speech, training=True)

In [43]:
# Model 2


In [45]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv1D,Conv2D, MaxPooling2D,MaxPooling1D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D,MaxPool1D,GlobalAveragePooling1D
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.activations import relu, softmax, swish
import tensorflow
import tensorflow as tf

In [58]:
model1 = get_LCNN_o_4_dr_1(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,wav2_input_shape, use_mel = use_mel, use_cqt = use_cqt, use_stft = use_stft, ord1 = ord1, dp = dp, fc = fc, ext = ext,use_wav2=use_wav2)

In [59]:

model2 = get_LCNN_o_4_dr_2(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,wav2_input_shape, use_mel = use_mel, use_cqt = use_cqt, use_stft = use_stft, ord1 = ord1, dp = dp, fc = fc, ext = ext,use_wav2=use_wav2)


In [60]:
model1.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 interval (InputLayer)          [(None, 229, 1)]     0           []                               
                                                                                                  
 conv1d_160 (Conv1D)            (None, 229, 9)       36          ['interval[0][0]']               
                                                                                                  
 conv1d_161 (Conv1D)            (None, 229, 9)       36          ['interval[0][0]']               
                                                                                                  
 maximum_152 (Maximum)          (None, 229, 9)       0           ['conv1d_160[0][0]',             
                                                                  'conv1d_161[0][0]']       

 conv1d_176 (Conv1D)            (None, 221, 9)       171         ['max_pooling1d_87[0][0]']       
                                                                                                  
 conv1d_177 (Conv1D)            (None, 221, 9)       171         ['max_pooling1d_87[0][0]']       
                                                                                                  
 maximum_160 (Maximum)          (None, 221, 9)       0           ['conv1d_176[0][0]',             
                                                                  'conv1d_177[0][0]']             
                                                                                                  
 max_pooling1d_88 (MaxPooling1D  (None, 220, 9)      0           ['maximum_160[0][0]']            
 )                                                                                                
                                                                                                  
 conv1d_17

                                                                                                  
 conv1d_184 (Conv1D)            (None, 217, 9)       171         ['max_pooling1d_91[0][0]']       
                                                                                                  
 conv1d_185 (Conv1D)            (None, 217, 9)       171         ['max_pooling1d_91[0][0]']       
                                                                                                  
 maximum_173 (Maximum)          (None, 384, 384, 32  0           ['conv2d_146[0][0]',             
                                )                                 'conv2d_147[0][0]']             
                                                                                                  
 batch_normalization_54 (BatchN  (None, 60, 157, 32)  96         ['maximum_182[0][0]']            
 ormalization)                                                                                    
          

 )                                                                                                
                                                                                                  
 batch_normalization_50 (BatchN  (None, 192, 192, 48  144        ['maximum_175[0][0]']            
 ormalization)                  )                                                                 
                                                                                                  
 conv2d_170 (Conv2D)            (None, 30, 79, 64)   27712       ['batch_normalization_56[0][0]'] 
                                                                                                  
 conv2d_171 (Conv2D)            (None, 30, 79, 64)   27712       ['batch_normalization_56[0][0]'] 
                                                                                                  
 conv1d_190 (Conv1D)            (None, 214, 64)      1216        ['max_pooling1d_94[0][0]']       
          

                                                                  'conv1d_195[0][0]']             
                                                                                                  
 maximum_178 (Maximum)          (None, 96, 96, 32)   0           ['conv2d_156[0][0]',             
                                                                  'conv2d_157[0][0]']             
                                                                                                  
 batch_normalization_58 (BatchN  (None, 15, 40, 32)  96          ['maximum_187[0][0]']            
 ormalization)                                                                                    
                                                                                                  
 max_pooling1d_97 (MaxPooling1D  (None, 211, 9)      0           ['maximum_169[0][0]']            
 )                                                                                                
          

                                                                                                  
 is_preg (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 max_pooling1d_99 (MaxPooling1D  (None, 209, 9)      0           ['maximum_171[0][0]']            
 )                                                                                                
                                                                                                  
 max_pooling2d_35 (MaxPooling2D  (None, 48, 48, 32)  0           ['maximum_180[0][0]']            
 )                                                                                                
                                                                                                  
 global_average_pooling2d_9 (Gl  (None, 32)          0           ['max_pooling2d_39[0][0]']       
 obalAvera

In [61]:
model2.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 interval (InputLayer)          [(None, 229, 1)]     0           []                               
                                                                                                  
 conv1d_200 (Conv1D)            (None, 229, 9)       36          ['interval[0][0]']               
                                                                                                  
 conv1d_201 (Conv1D)            (None, 229, 9)       36          ['interval[0][0]']               
                                                                                                  
 maximum_190 (Maximum)          (None, 229, 9)       0           ['conv1d_200[0][0]',             
                                                                  'conv1d_201[0][0]']       

 conv1d_216 (Conv1D)            (None, 221, 9)       171         ['max_pooling1d_107[0][0]']      
                                                                                                  
 conv1d_217 (Conv1D)            (None, 221, 9)       171         ['max_pooling1d_107[0][0]']      
                                                                                                  
 maximum_198 (Maximum)          (None, 221, 9)       0           ['conv1d_216[0][0]',             
                                                                  'conv1d_217[0][0]']             
                                                                                                  
 max_pooling1d_108 (MaxPooling1  (None, 220, 9)      0           ['maximum_198[0][0]']            
 D)                                                                                               
                                                                                                  
 conv1d_21

                                                                                                  
 conv1d_224 (Conv1D)            (None, 217, 9)       171         ['max_pooling1d_111[0][0]']      
                                                                                                  
 conv1d_225 (Conv1D)            (None, 217, 9)       171         ['max_pooling1d_111[0][0]']      
                                                                                                  
 maximum_211 (Maximum)          (None, 384, 384, 32  0           ['conv2d_182[0][0]',             
                                )                                 'conv2d_183[0][0]']             
                                                                                                  
 batch_normalization_66 (BatchN  (None, 60, 157, 32)  96         ['maximum_220[0][0]']            
 ormalization)                                                                                    
          

 D)                                                                                               
                                                                                                  
 batch_normalization_62 (BatchN  (None, 192, 192, 48  144        ['maximum_213[0][0]']            
 ormalization)                  )                                                                 
                                                                                                  
 conv2d_206 (Conv2D)            (None, 30, 79, 64)   27712       ['batch_normalization_68[0][0]'] 
                                                                                                  
 conv2d_207 (Conv2D)            (None, 30, 79, 64)   27712       ['batch_normalization_68[0][0]'] 
                                                                                                  
 conv1d_230 (Conv1D)            (None, 214, 64)      1216        ['max_pooling1d_114[0][0]']      
          

                                                                  'conv1d_235[0][0]']             
                                                                                                  
 maximum_216 (Maximum)          (None, 96, 96, 32)   0           ['conv2d_192[0][0]',             
                                                                  'conv2d_193[0][0]']             
                                                                                                  
 batch_normalization_70 (BatchN  (None, 15, 40, 32)  96          ['maximum_225[0][0]']            
 ormalization)                                                                                    
                                                                                                  
 max_pooling1d_117 (MaxPooling1  (None, 211, 9)      0           ['maximum_207[0][0]']            
 D)                                                                                               
          

                                                                                                  
 is_preg (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 max_pooling1d_119 (MaxPooling1  (None, 209, 9)      0           ['maximum_209[0][0]']            
 D)                                                                                               
                                                                                                  
 max_pooling2d_43 (MaxPooling2D  (None, 48, 48, 32)  0           ['maximum_218[0][0]']            
 )                                                                                                
                                                                                                  
 global_average_pooling2d_11 (G  (None, 32)          0           ['max_pooling2d_47[0][0]']       
 lobalAver

In [62]:
n_epoch = 5
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 8

if mixup :
    beta_param = .7
else :
    beta_param = 0
    


params = {'batch_size': batch_size,
          #          'input_shape': (100, 313, 1),
          'shuffle': True,
          'chaug': chaug,
          'beta_param': beta_param,
          'cout': cout
#              'mixup': mixup,
          #          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          #          'highpass': [.5, [78,79,80,81,82,83,84,85]]
#              'ranfilter2' : [3, [18,19,20,21,22,23]]
          #           'dropblock' : [30, 100]
          #'device' : device
}

if mixup :
    params['mixup'] = mixup
    params['ranfilter2'] = ranfil
else :
    params['cutout'] = cout

params_no_shuffle = {'batch_size': batch_size,
                     #          'input_shape': (100, 313, 1),
                     'shuffle': False,
                     'beta_param': 0.7,
                     'mixup': False
                     #'device': device
}


# TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
#             features_trn['interval'],features_trn['wav2'],features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
#            features_trn['mm_labels'],  ## our Y
#                         **params)()


# ValidDGen_1 = Generator0([features_test['age'],features_test['sex'], features_test['hw'], 
#                               features_test['preg'], features_test['loc'],features_test['interval'],
#                               features_test['wav2'], features_test['mel1'],features_test['cqt1'],features_test['stft1']], 
#                              features_test['mm_labels'],  ## our Y
                         
#                          **params)()





if ord1 :
    class_weight = {0: mm_weight, 1: 1}
else :
    class_weight = {0: mm_weight, 1: wunknown, 2:1}

    
    
    
    

if mixup :
    
    
    TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], 
                              features_trn['preg'], features_trn['loc'],features_trn['interval'],
                              features_trn['wav2'],features_trn['mel1'],
                              features_trn['cqt1'],
                              features_trn['stft1']],features_trn['mm_labels'],**params)()
    
    model1.fit(TrainDGen_1,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'],
                                   
                                   features_test['preg'], features_test['loc'],features_test['interval']
                                   ,features_test['wav2'],features_test['mel1'],
                                   features_test['cqt1'],
                                   features_test['stft1']],features_test['mm_labels']),
               
               callbacks=[lr,model1_callback],
               steps_per_epoch=np.ceil(len(features_trn['mm_labels'])/batch_size),
               class_weight=class_weight, 
               epochs = n_epoch)
    
    
 

else :
    TrainGen = DataGenerator([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
                      features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                     mm_lbs_trn,  ## our Y
                     **params)
    model1.fit(TrainGen,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                                   features_test['preg'], features_test['loc'], features_test['mel1'], 
                                   features_test['cqt1'], features_test['stft1']], 
                                  mm_lbs_test), 
               callbacks=[lr],
                      steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
               class_weight=class_weight, 
               epochs = n_epoch)



Epoch 1/5
317/317 [==============================] - 190s 582ms/step - loss: 1.0447 - accuracy: 0.6952 - auc: 0.7531 - val_loss: 0.5081 - val_accuracy: 0.7813 - val_auc: 0.8380 - lr: 9.9851e-04
Epoch 2/5
317/317 [==============================] - 177s 559ms/step - loss: 0.8835 - accuracy: 0.7670 - auc: 0.8279 - val_loss: 0.7897 - val_accuracy: 0.4057 - val_auc: 0.4147 - lr: 9.8036e-04
Epoch 3/5
317/317 [==============================] - 179s 566ms/step - loss: 0.8546 - accuracy: 0.7859 - auc: 0.8522 - val_loss: 0.4532 - val_accuracy: 0.8209 - val_auc: 0.8925 - lr: 7.8798e-04
Epoch 4/5
317/317 [==============================] - 179s 564ms/step - loss: 0.8473 - accuracy: 0.8036 - auc: 0.8705 - val_loss: 0.8712 - val_accuracy: 0.3344 - val_auc: 0.3615 - lr: 2.2202e-04
Epoch 5/5
317/317 [==============================] - 180s 567ms/step - loss: 0.8332 - accuracy: 0.8095 - auc: 0.8743 - val_loss: 0.4569 - val_accuracy: 0.8336 - val_auc: 0.9056 - lr: 2.9642e-05


In [63]:
n_epoch = 5
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))

batch_size = 8


params = {'batch_size': batch_size,
          #          'input_shape': (100, 313, 1),
          'shuffle': True,
          'chaug': 0,
          'beta_param': beta_param,
          'cout': cout,
#              'mixup': True,
          #          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#            'highpass': [.5, [78,79,80,81,82,83,84,85]],
#              'ranfilter2' : [3, [18,19,20,21,22,23]]
        #           'dropblock' : [30, 100]
          #'device' : device
}



if mixup :
    params['mixup'] = mixup
    params['ranfilter2'] = ranfil
else :
    params['cutout'] = cout


params_no_shuffle = {'batch_size': batch_size,
                     #          'input_shape': (100, 313, 1),
                     'shuffle': False,
                     'beta_param': 0.7,
                     'mixup': False
                     #'device': device
}




if mixup :

    
    class_weight = {0: oo_weight, 1: 1.}
    TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'],
                              features_trn['loc'],features_trn['interval'],features_trn['wav2'],
                              features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']],
                             features_trn['out_labels'], **params)()

    model2.fit(TrainDGen_1,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'],
                                   features_test['preg'], features_test['loc'],features_test['interval'],
                                   features_test['wav2'],features_test['mel1'],
                                   features_test['cqt1'],
                                   features_test['stft1']],features_test['out_labels']), 
               
               callbacks=[lr,model2_callback],
               steps_per_epoch=np.ceil(len(features_trn['out_labels'])/batch_size),
               class_weight=class_weight, 
               epochs = n_epoch)
    
else :
    TrainGen = DataGenerator([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
                      features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                     out_lbs_trn,  ## our Y
                     **params)
    model2.fit(TrainGen,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'],
                                   features_test['preg'], features_test['loc'],features_test['interval'],
                                   features_test['wav2'], features_test['mel1'],features_test['cqt1'],
                                   features_test['stft1']],features_test['out_labels']), 
               callbacks=[lr],
               class_weight=class_weight, 
               epochs = n_epoch)


















Epoch 1/5
317/317 [==============================] - 187s 574ms/step - loss: 1.3874 - accuracy: 0.5039 - auc: 0.5012 - val_loss: 0.6470 - val_accuracy: 0.6260 - val_auc: 0.6664 - lr: 9.9851e-04
Epoch 2/5
317/317 [==============================] - 179s 566ms/step - loss: 1.2658 - accuracy: 0.4878 - auc: 0.5028 - val_loss: 0.7134 - val_accuracy: 0.5024 - val_auc: 0.5932 - lr: 9.8036e-04
Epoch 3/5
317/317 [==============================] - 179s 566ms/step - loss: 1.2305 - accuracy: 0.4795 - auc: 0.5041 - val_loss: 0.6577 - val_accuracy: 0.5721 - val_auc: 0.6235 - lr: 7.8798e-04
Epoch 4/5
317/317 [==============================] - 178s 561ms/step - loss: 1.2282 - accuracy: 0.4929 - auc: 0.5192 - val_loss: 0.7032 - val_accuracy: 0.5246 - val_auc: 0.6131 - lr: 2.2202e-04
Epoch 5/5
317/317 [==============================] - 179s 566ms/step - loss: 1.2082 - accuracy: 0.4874 - auc: 0.5263 - val_loss: 0.6803 - val_accuracy: 0.5261 - val_auc: 0.6131 - lr: 2.9642e-05


In [64]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 120,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen1': 246000,
 'min_dist': 500,
 'max_interval_len': 229,
 'trim': 1,
 'use_mel': True,
 'use_cqt': False,
 'use_stft': False,
 'use_interval': True,
 'use_wav2': True,
 'per_sec': 4000,
 'ord1': True,
 'mm_mean': False,
 'dp': 0,
 'fc': False,
 'ext': True,
 'oo_weight': 3,
 'mm_weight': 3,
 'chaug': 10,
 'cout': 0.8,
 'wunknown': 1,
 'mixup': True,
 'n1': 0}

In [65]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape
params_feature['interval_input_shape'] = interval_input_shape
params_feature['wav2_input_shape'] = wav2_input_shape

In [66]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 120,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen1': 246000,
 'min_dist': 500,
 'max_interval_len': 229,
 'trim': 1,
 'use_mel': True,
 'use_cqt': False,
 'use_stft': False,
 'use_interval': True,
 'use_wav2': True,
 'per_sec': 4000,
 'ord1': True,
 'mm_mean': False,
 'dp': 0,
 'fc': False,
 'ext': True,
 'oo_weight': 3,
 'mm_weight': 3,
 'chaug': 10,
 'cout': 0.8,
 'wunknown': 1,
 'mixup': True,
 'n1': 0,
 'mel_shape': (120, 313, 1),
 'cqt_shape': (1, 1, 1),
 'stft_shape': (1, 1, 1),
 'interval_input_shape': (229, 1),
 'wav2_input_shape': (246000,)}

In [67]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2
    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [68]:
model_folder

'lcnn2'

In [69]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'lcnn1', m_name2 = 'lcnn2', param_feature = params_feature)

1

In [86]:
model

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 120,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen1': 246000,
 'min_dist': 500,
 'max_interval_len': 229,
 'trim': 1,
 'use_mel': True,
 'use_cqt': False,
 'use_stft': False,
 'use_interval': True,
 'use_wav2': True,
 'per_sec': 4000,
 'ord1': True,
 'mm_mean': False,
 'dp': 0,
 'fc': False,
 'ext': True,
 'oo_weight': 3,
 'mm_weight': 3,
 'chaug': 10,
 'cout': 0.8,
 'wunknown': 1,
 'mixup': True,
 'n1': 0,
 'mel_shape': (120, 313, 1),
 'cqt_shape': (1, 1, 1),
 'stft_shape': (1, 1, 1),
 'interval_input_shape': (229, 1),
 'wav2_input_shape': (246000,),
 'model1': 'lcnn1',
 'model2': 'lcnn2',
 'model_fnm1': 'lcnn2/lcnn1_model1.hdf5',
 'model_fnm2': 'lcnn2/lcnn2_model2.hdf5'}

In [70]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)

    return info_m

In [231]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']

##########################################################################    
    if model['model1'] == 'lcnn1' :
        model1 = get_LCNN_o_4_dr_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], model['interval_input_shape'],model['wav2_input_shape']
                                 , use_mel = model['use_mel'],use_cqt = model['use_cqt'], use_stft = model['use_stft'], 
                                 ord1 = model['ord1'], dp = model['dp'], fc = model['fc'], ext = model['ext'],use_wav2=model['use_wav2'])    
    if model['model1'] == 'lcnn2' :
        model1 = get_LCNN_o_4_dr_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], model['interval_input_shape'],model['wav2_input_shape']
                                 , use_mel = model['use_mel'],use_cqt = model['use_cqt'], use_stft = model['use_stft'], 
                                 ord1 = model['ord1'], dp = model['dp'], fc = model['fc'], ext = model['ext'],use_wav2=model['use_wav2'])   
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
############################################################################ 


    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    trim = model['trim']
    use_mel = model['use_mel']
    use_cqt = model['use_cqt']
    use_stft = model['use_stft']
    use_interval = model['use_interval']
    use_wav2 = model['use_wav2']
    maxlen1 = model['maxlen1']
    min_dist = model['min_dist']
    max_interval_len = model['max_interval_len']
    per_sec = model['per_sec']
#    use_raw = model['use_raw']
    

    features['interval'] = []
    tmp_interval=[]  
    
    features['wav2'] = []
    tmp_wav=[]
    tmp_total_interval = []  
    
   
    for i in range(len(recordings)):
        data = recordings[i]
        tmp_wav.append(data)
        
    padded =pad_sequences(tmp_wav, maxlen=maxlen1, dtype='float64', padding='post', truncating='post', value=0.0)
    
    padded=np.array(padded, dtype=np.float32)
    features['wav2']=padded
    features['wav2'] = np.array(features['wav2'])
    
    if use_interval:
        for i in tqdm.tqdm(range(len(recordings))) :
            datos=recordings[i]
            filtros=sio.loadmat('./Filters1')
            tmp_interval = []
            n_samp = len(datos)//per_sec
                               
                
            try:
                for k in range(n_samp):
                    X = datos[k*per_sec:(k+1)*per_sec]
                    Fs= per_sec
                    Fpa20=filtros['Fpa20'];			        # High pass filter
                    Fpa20=Fpa20[0];					# High pass filter
                    Fpb100=filtros['Fpb100'];		        # Low-pass Filter
                    Fpb100=Fpb100[0];				# Low-pass Filter
                    Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
                    Xf=vec_nor(Xf);			
            
            # Derivate of the Signal
                    dX=derivate(Xf);				# Derivate of the signal
                    dX=vec_nor(dX);					# Vector Normalizing
            # Square of the signal
                    dy=np.square(Xf);
                    dy=vec_nor(dy);
                    
                    size=np.shape(Xf)				# Rank or dimension of the array
                    fil=size[0];					# Number of rows

                    positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
                    positive=positive[0];                           # Getting the Vector

                    points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
                    points=points[0];                               # Getting the point vector

                    peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
                    peaks=peaks[0];                                 # Getting the point vector

            
                    for i in range(0,fil):
                        if dX[i]>0:
                            positive[i]=1;
                        else:
                            positive[i]=0;
    
                    for i in range(0,fil):
                        if (positive[i]==1 and positive[i+1]==0):
                            points[i]=Xf[i];
                        else:
                            points[i]=0;

                    indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=min_dist);
                    lenght=np.shape(indexes)			# Get the length of the index vector		
                    lenght=lenght[0];				# Get the value of the index vector

                    for i in range(0,lenght):
                        p=indexes[i];
                        peaks[p]=points[p];
        
                    n=np.arange(0,fil);                            # Vector to the X axes (Number of Samples)
                    indexes =indexes+(k*per_sec)    
                    tmp_peaks = np.array(indexes)
                    
                    
                    tmp_interval.extend(tmp_peaks)
                    
                tmp_interval = np.array(tmp_interval)
                tmp_interval = np.diff(tmp_interval)
                    
                tmp_total_interval.append(tmp_interval)
 
           
            except:
                print(filename)
                tmp_peaks = np.zeros(max_interval_len)
                tmp_total_interval.append(tmp_peaks)
                    
    else :
                        
        tmp_peaks = np.zeros(max_interval_len)
        tmp_total_interval.append(tmp_peaks)       
                    
     
    padded =pad_sequences(tmp_total_interval, maxlen=max_interval_len, dtype='float64', padding='post', truncating='post', value=0.0)
        
    for i in range(len(padded)):
        features['interval'].append(padded[i])
        
        
    for i in range(len(features['interval'])):
        features['interval'][i] = np.array(features['interval'][i])

    if use_interval:
        
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)
        
    else:
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)        
    

    features['interval']=np.array(features['interval'])
    
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        if use_mel :
            mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        else :
            mel1 = np.zeros( (1,1) )
            
        features['mel1'].append(mel1)
        
    M, N = features['mel1'][0].shape
    
    if use_mel :
        for i in range(len(features['mel1'])) :
            features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    
    features['mel1'] = np.array(features['mel1'])

    
  
    features['cqt1'] = []
    for i in range(len(recordings)) :
        if use_cqt :
            mel1 = feature_extract_cqt(recordings[i], samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        else:
            mel1 = np.zeros( (1,1,1) )
            
        features['cqt1'].append(mel1)
        
    M, N,__ = features['cqt1'][0].shape
    
    if use_cqt :
        for i in range(len(features['cqt1'])) :
            features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    
    features['cqt1'] = np.array(features['cqt1'])
    
    
    features['stft1'] = []
    for i in range(len(recordings)) :
        if use_stft :
            mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                        win_length = win_length)[0]
        else :
            mel1 = np.zeros( (1,1,1) )
        
        features['stft1'].append(mel1)
        
    M, N,__ = features['stft1'][0].shape
    if use_stft :
        for i in range(len(features['stft1'])) :
            features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])
    
#     return features       

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], 
                           features['loc'], features['interval'],features['wav2'],features['mel1'], features['stft1'], features['cqt1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], 
                           features['loc'], features['interval'],features['wav2'],features['mel1'], features['stft1'], features['cqt1']])
    if model['ord1'] :
        idx1 = res1.argmax(axis=0)[0]
        murmur_p = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
        murmur_probabilities = np.zeros((3,))
        murmur_probabilities[0] = murmur_p[0]
        murmur_probabilities[1] = 0
        murmur_probabilities[2] = murmur_p[1]
        outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
    else :
        if model['mm_mean'] :
            murmur_probabilities = res1.mean(axis = 0)
        else :
            idx1 = res1.argmax(axis=0)[0]
            murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
        outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기

        
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    
    
    if murmur_probabilities[0] > 0.482 :
        idx = 0
    else :
        idx = 2
    murmur_labels[idx] = 1

    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    if outcome_probabilities[0] > 0.607 :
        idx = 0
    else :
        idx = 1    
        # idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

In [232]:
model_folder

'lcnn2'

In [233]:
data_folder = test_folder

In [234]:
data_folder

'/home/jk21/Downloads/Data/data/murmur/test'

In [235]:
output_folder ='/home/jk21/Documents/hmd/jk_classifier/out_lcnn2'

In [236]:
allow_failures=True

In [237]:
verbose=1

In [238]:
model = load_challenge_model(model_folder, verbose)

In [239]:
patient_files = find_patient_files(data_folder)

In [240]:
num_patient_files = len(patient_files)

In [241]:
patient_data = load_patient_data(patient_files[0])

In [242]:
patient_data

'33151 4 4000\nAV 33151_AV.hea 33151_AV.wav 33151_AV.tsv\nPV 33151_PV.hea 33151_PV.wav 33151_PV.tsv\nTV 33151_TV.hea 33151_TV.wav 33151_TV.tsv\nMV 33151_MV.hea 33151_MV.wav 33151_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 141.0\n#Weight: 30.9\n#Pregnancy status: False\n#Murmur: Present\n#Murmur locations: MV+TV\n#Most audible location: TV\n#Systolic murmur timing: Holosystolic\n#Systolic murmur shape: Plateau\n#Systolic murmur grading: I/VI\n#Systolic murmur pitch: Low\n#Systolic murmur quality: Harsh\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [243]:
recordings = load_recordings(data_folder, patient_data)

In [244]:
classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose)

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  8.31it/s]


In [245]:
classes

['Present', 'Unknown', 'Absent', 'Abnormal', 'Normal']

In [246]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose=1):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [247]:
test_folder

'/home/jk21/Downloads/Data/data/murmur/test'

In [248]:
output_folder = '/home/jk21/Documents/hmd/jk_classifier/out_lcnn2'

In [249]:
model_folder

'lcnn2'

In [250]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 2)

Loading Challenge model...
Running model on Challenge data...
    1/191...


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  8.22it/s]


    2/191...


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  8.00it/s]


    3/191...


...

    190/191...


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  5.36it/s]


    191/191...


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  4.67it/s]


Done.


In [251]:
from evaluate_model import *



In [252]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.699,0.533,0.498,0.791,0.677,18390.927

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.752,0.759,0.661,0.675,0.798,10128.482

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.801,0.500,0.795
AUPRC,0.623,0.073,0.901
F-measure,0.617,0.000,0.878
Accuracy,0.658,0.000,0.906

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.752,0.752
AUPRC,0.779,0.738
F-measure,0.730,0.592
Accuracy,0.857,0.484



In [253]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.4067772331974269
0.5932227697330698
0.6537978667863377
0.34620212931283484


In [254]:
for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")

threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.699,0.533,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.752,0.759,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.801,0.500,0.795
AUPRC,0.623,0.073,0.901
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.752,0.752
AUPRC,0.779,0.738
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.699,0.533,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.752,0.759,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.801,0.500,0.795
AUPRC,0.623,0.073,0.901
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU